In [15]:
# FIXME: I'm probably leaking my IP address all over the place ...

In [10]:
%load_ext autoreload
%autoreload 2

# Reading Version Message Payloads

In the last lesson we encountered the Bitcoin protocol's [Version Handshake](https://en.bitcoin.it/wiki/Version_Handshake). We saw how Bitcoin network peers will only converse with us if we first introduce ourselves with a `version` message.

But _we cheated_. I gave you a serialized `version` message and didn't tell you how I created it.

_We were lazy_: we didn't parse the cryptic `payload` of the `version` message that our peer sent us.

_We were rude_! After listening for our peer's `version` message we stopped listening and never received or responded to their `verack` message -- completing the handshake. Our peer was left hanging ...

So you see, we have much to fix!

### Housekeeping

Last time we created a `NetworkEnvelope` class. I'm going to throw that away and use functions and dictionaries instead. Simpler this way!

Here's where we are:

In [8]:
from hashlib import sha256

NETWORK_MAGIC = b'\xf9\xbe\xb4\xd9'

def double_sha256(s):
    return sha256(sha256(s).digest()).digest()

def deserialize_message(stream):
    """ payload attributes at top level """
    msg = {}
    magic = stream.read(4)
    if magic != NETWORK_MAGIC:
        raise Exception(f'Magic is wrong: {magic}')
    msg['command'] = stream.read(12).strip(b'\x00')
    payload_length = int.from_bytes(stream.read(4), 'little')
    checksum = stream.read(4)
    msg['payload'] = stream.read(payload_length)
    calculated_checksum = double_sha256(msg['payload'])[:4]
    if calculated_checksum != checksum:
        raise Exception('Checksum does not match')
    return msg


In [9]:
import socket

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

sock = socket.socket()
sock.connect(("35.198.151.21", 8333))
stream = sock.makefile('rb')

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
msg = deserialize_message(stream)

print(msg)
print(msg['command'])
print(msg['payload'])

{'command': b'version', 'payload': b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x0028j\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffFqPG\xa8\xc6\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00!\xf8\xe8\xff\xceL+s\x10/Satoshi:0.16.3/U\x99\x08\x00\x01'}
b'version'
b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x0028j\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffFqPG\xa8\xc6\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00!\xf8\xe8\xff\xceL+s\x10/Satoshi:0.16.3/U\x99\x08\x00\x01'



The payload of our message is still `bytes`. We need to decode the payload in the same way that we decoded the outer message structure itself. In order to do will apply exactly the same skills we started to learn last time in order to 

# Objective: Interpret the Message Payload

Here's how payloads look now:

```
b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x0028j\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffFqPG\xa8\xc6\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00!\xf8\xe8\xff\xceL+s\x10/Satoshi:0.16.3/U\x99\x08\x00\x01'
```

In this lesson we'll learn to interpret these bytes as a dictionary like this:

```
{'latest_block': 563541,
 'nonce': 8298811190300702753,
 'receiver_address': {'ip': '::ffff:70.113.80.71',
                      'port': 43206,
                      'services': 0},
 'relay': 1,
 'sender_address': {'ip': '0.0.0.0', 'port': 0, 'services': 1037},
 'services': 1037,
 'timestamp': 1550465074,
 'user_agent': b'/Satoshi:0.16.3/',
 'version': 70015}
```

In [14]:
# FIXME: kill

from io import BytesIO
from pprint import pprint
from proto import deserialize_version_payload

pprint(deserialize_version_payload(BytesIO(msg['payload'])))

{'latest_block': 563541,
 'nonce': 8298811190300702753,
 'receiver_address': {'ip': '::ffff:70.113.80.71',
                      'port': 43206,
                      'services': 0},
 'relay': 1,
 'sender_address': {'ip': '0.0.0.0', 'port': 0, 'services': 1037},
 'services': 1037,
 'timestamp': 1550465074,
 'user_agent': b'/Satoshi:0.16.3/',
 'version': 70015}


### Some Refactoring

Almost the same as last lesson except for:
* I renamed `Message` to `Packet` because we're going to start defining things like `VersionMessage` and `VerackMessage` and so this seemed a little more clear.
* `Packet` instances look like pretty tables when printed. This is because I added a `Network.__str__` method to [ibd/one/complete.py](./ibd/one/complete.py). `object.__str__` is the function Python calls when determining _how_ to print a given `object`. This `Packet.__str__` method simply runs a few of its values through this [tabulate](https://bitbucket.org/astanin/python-tabulate) program, which I added to our requirements.txt file.

### The Payload

Our next task is to parse this payload. Besides the "/Satoshi:0.16.3/" -- clearly a user agent -- the rest of the payload isn't human readable.

But have no fear -- we will decode the message payload in the same manner as we decoded the overall message structure in our `deserialize_message` method.

[This chart](https://en.bitcoin.it/wiki/Protocol_documentation#version) from the protocol documentation will act as our blueprint:

![image](../images/version-message.png)

### Old Types

Here we encounter some "types" we are now familiar with from the first lesson -- `int32_t` / `uint64_t` / `int64_t` -- which are different types in a "low-level" language like C++, but are all equivalent to the `int` type in Python. Our previously implemented `bytes_to_int` (FIXME) can handle these just fine.

### New Types

But we also encounter some new types: `net_addr`, `varstr`, and `bool`. 

Even worse, if we click on the [`varstr` link](https://en.bitcoin.it/wiki/Protocol_documentation#Variable_length_string) we see that it contains one additional type: `varint`. 

Worse still, the [`net_addr` link](https://en.bitcoin.it/wiki/Protocol_documentation#Network_address) contains `time`, `services`, `ip` and `port` fields nominally of types `uint32`, `uint64_t` and `char[16]` but in order for us to make sense of what they hell them mean each requires parsing: the `time` integer as a Unix timestamp, the `services` integer as a "bitfield" (whatever that is!), and IP address can be either IPv4 or IPv6 and our code must be able to tell the difference! Also, the `port` field is "network byte-order" according to the wiki.

Oh, and remember how I mentioned that Satoshi usually, but not always, encoded his integers in "little endian" byte order (least significant digits is on the left)? Well, the `port` attribute of `net_addr` is encoded "big endian", where the *most* significant digit is on the left. Yes, the exact opposite of everything else!!!

Hunker down for a looooooong lesson!